In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
for i in df_train.columns:
    if df_train[i].dtype == object:
        label = LabelEncoder()
        df_train[i] = label.fit_transform(df_train[i])

In [5]:
for i in df_test.columns:
    if df_test[i].dtype == object:
        label = LabelEncoder()
        df_test[i] = label.fit_transform(df_test[i])

In [6]:
df_test = df_test.drop(["id" , "CustomerId", "Surname", "Tenure" , "HasCrCard" , "EstimatedSalary"] , axis =1 )
df_train = df_train.drop(["id" , "CustomerId", "Surname", "Tenure" , "HasCrCard" , "EstimatedSalary"] , axis =1 )

In [7]:
X = df_train.drop("Exited" ,axis = 1)
y = df_train["Exited"]

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

In [9]:
X = scaler.fit_transform(X)

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
xgb_m = XGBClassifier()

In [19]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

In [20]:
full_model = GridSearchCV(estimator=xgb_m, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

In [21]:
full_model.fit(X, y)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [22]:
y_pred = full_model.predict(df_test)

In [23]:
df_sub= pd.read_csv('sample_submission.csv')

In [24]:
df_sub["Exited"] = y_pred

In [25]:
df_sub.to_csv('Tiko_spam_xg.csv', index=False, index_label=False)